작업 엔진 생성

In [1]:
import sys
sys.path.append('/home/ubuntu/Pauli-propagation-surrogate/src')
from cuquantum.bindings  import cupauliprop as cupp
import easy_cuQU as ezcQ
import numpy as np
import cupy as cp
from types import SimpleNamespace

# 양자 게이트 시퀀스 정의
ctx = SimpleNamespace()


print("✓ cuPauliProp 라이브러리 로드 완료")
print(f"  NumPy: {np.__version__}")
print(f"  CuPy: {cp.__version__}")
print(f"  easy_cuQU: {ezcQ.__version__}")

NUM_QUBITS = 3
num_packed_ints = ezcQ.get_packed_ints(NUM_QUBITS)
ctx.num_qubits = NUM_QUBITS
ctx.num_packed_ints = num_packed_ints
print()

# user_friendly_terms = [ (0.8,  "X0 X2 Z1"),
#                         (-0.6, "X4 Z3"),
#                         (0.5,  "Z0 Z1 Z4"),
#                         (0.4,  "Y2 Y3"),
#                         (-0.3, "X6 Z5"),
#                         (0.9,  "X7 Z6"),]
user_friendly_terms = [ (1,  "Z0 Z1 Z2")]

OBS_TERMS = ezcQ.create_observable(user_friendly_terms, NUM_QUBITS)
ctx.obs_terms = OBS_TERMS
print()

d_input_pauli, d_input_coef = ezcQ.observable_to_cuQU_input(OBS_TERMS)
ctx.d_input_pauli = d_input_pauli
ctx.d_input_coef = d_input_coef
print()

ezcQ.cleanup_cupauli()
handle = ezcQ.make_handle()
ctx.handle = handle
print()

# max_terms, (d_input_pauli_buffer, d_input_coef_buffer, d_output_pauli_buffer, d_output_coef_buffer) = ezcQ.create_buffer(NUM_QUBITS, d_input_pauli, d_input_coef, max_terms= 10000)
max_terms, (d_input_pauli_buffer, d_input_coef_buffer, d_output_pauli_buffer, d_output_coef_buffer) = ezcQ.create_buffer(ctx=ctx)
ctx.max_terms = max_terms
ctx.d_input_pauli_buffer = d_input_pauli_buffer   ; ctx.d_input_coef_buffer = d_input_coef_buffer
ctx.d_output_pauli_buffer = d_output_pauli_buffer ; ctx.d_output_coef_buffer = d_output_coef_buffer
# print()

# workspace = ezcQ.create_workspace(handle, d_input_pauli_buffer, d_input_coef_buffer, d_output_pauli_buffer, d_output_coef_buffer, frac = 0.8)
workspace = ezcQ.create_workspace(ctx=ctx, frac = 0.8)
ctx.workspace = workspace
# print()

# input_expansion, output_expansion = ezcQ.reset_expansions(handle, NUM_QUBITS, d_input_pauli_buffer, d_input_coef_buffer, d_output_pauli_buffer, d_output_coef_buffer=d_output_coef_buffer)
input_expansion, output_expansion = ezcQ.reset_expansions(ctx=ctx)
ctx.input_expansion = input_expansion
ctx.output_expansion = output_expansion

✓ cuPauliProp 라이브러리 로드 완료
  NumPy: 2.3.5
  CuPy: 13.6.0
  easy_cuQU: 0.1.0
큐비트 수: 3, 필요한 packed integers 수: 1

당신이 정의한 관측자의 mask 표현:
Term 0: coef=+1.0, X_mask=0b0000000000, Z_mask=0b0000000111

당신의 관측자가 cuPauliProp 입력 형식으로 변환됩니다

리소스 정리 중...
✓ 모든 리소스 해제 완료. 새로운 시작을 위해 준비되었습니다.
cuPauliProp 핸들 생성 완료.

큐비트 수: 3, 필요한 packed integers 수: 1
max terms의 기본 값은 10000개 입니다.
max_terms 설정값: 10000
✓ 메모리 풀 비움: 현재 가용 14.47 GB / 총 14.58 GB
workspace를 할당합니다.
가용 메모리 14.47 GB의 80%인 11.57 GB를 사용합니다


In [2]:
from easy_cuQU import CircuitBuilder

circuit = CircuitBuilder(handle, NUM_QUBITS)
theta = np.pi/2
circuit.h(0)

circuit.cnot(0, 1)
circuit.cnot(0, 2)
# circuit.cnot(0, 3)
gate_ops = circuit.build()
ctx.gate_ops = gate_ops

✓ 3개 gate operators 생성 완료


In [3]:
# 1. Pauli 관측자 진화
final_exp, n_terms, input_exp, output_exp = ezcQ.observable_evolution(ctx=ctx)
# final_exp, n_terms, input_exp, output_exp = ezcQ.observable_evolution(
#     handle=handle,
#     gate_ops=gate_ops,
#     input_expansion=input_expansion,
#     output_expansion=output_expansion,
#     workspace=workspace,
#     d_input_pauli_buffer=d_input_pauli_buffer,
#     d_input_coef_buffer=d_input_coef_buffer,
#     d_output_pauli_buffer=d_output_pauli_buffer,
#     d_output_coef_buffer=d_output_coef_buffer,
#     truncation_strategies=None  # 필요시
# )

# 2. 기댓값 계산
expectation = ezcQ.compute_expectation(ctx=ctx)
# expectation = ezcQ.compute_expectation(
#     handle=handle,
#     evolved_expansion=final_exp,
#     final_num_terms=n_terms,
#     input_expansion=input_exp,
#     output_expansion=output_exp,
#     num_packed_ints=num_packed_ints,
#     num_qubits=NUM_QUBITS,
#     workspace=workspace,
#     d_input_pauli_buffer=d_input_pauli_buffer,
#     d_input_coef_buffer=d_input_coef_buffer,
#     d_output_pauli_buffer=d_output_pauli_buffer,
#     d_output_coef_buffer=d_output_coef_buffer
# )


Gate 3 (reverse): 1 input terms
  → 1 output terms
    Term 0: X=0b000 Z=0b110 coef=+1.0000

Gate 2 (reverse): 1 input terms
  → 1 output terms
    Term 0: X=0b000 Z=0b111 coef=+1.0000

Gate 1 (reverse): 1 input terms
  → 1 output terms
    Term 0: X=0b001 Z=0b110 coef=+1.0000

✓ Propagation 완료: 최종 1개 Pauli terms
총 소요 시간: 0.11 초
최종 Pauli Expansion 상태:
  Term 0: X=001 Z=110 coef=1.000000

결과: ⟨ψ|O|ψ⟩ = 0.000000


## Dummy  
아래 코드는 Pauli Propagation의 easy module을 만들기 전, local에서 실행한 코드 흔적임

### 이게 원래의 propagation code

In [ ]:
from cuquantum.bindings import cupauliprop as cupp
import time
print("Pauli propagation (역순, adjoint):")
print("Observable을 circuit의 역방향으로 propagate...\n")


# 매 실행마다 expansion을 원본으로 리셋
input_expansion, output_expansion = ezcQ.reset_expansions(handle, NUM_QUBITS, d_input_pauli_buffer, d_input_coef_buffer, d_output_pauli_buffer, d_output_coef_buffer)


def observable_evolution(handle, workspace, gate_ops, input_expansion=None, output_expansion=None, truncation_strategies=None):

    current_input = input_expansion
    current_output = output_expansion

    # 역순으로 gate 적용
    s = time.time()
    for gate_idx, gate in enumerate(reversed(gate_ops)):
        gate_number = len(gate_ops) - gate_idx
        
        # 현재 term 개수 확인
        num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)
        print(f"Gate {gate_number} (reverse): {num_terms} input terms")
        
        # View 생성
        input_view = cupp.pauli_expansion_get_contiguous_range(
            handle, current_input, 0, num_terms
        )
        
        # Operator 적용: G† O G
        # ⚠️ truncation은 나중에 추가 예정
        cupp.pauli_expansion_view_compute_operator_application(
            handle,
            input_view,
            current_output,
            gate,
            1,  # adjoint=1 → G† O G
            1,  # sort=True → 정렬하여 중복 term 찾기
            0,  # keep_duplicates=False → 중복 term 병합 (계수 합산)
            0,  # num_truncation_strategies=0 (truncation 비활성화)
            truncation_strategies,  # truncation_strategies=None
            workspace  # workspace 포인터 주소
        )
        
        # View 삭제
        cupp.destroy_pauli_expansion_view(input_view)
        
        # Input/Output swap
        current_input, current_output = current_output, current_input
        
        # 진행 상황
        new_num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)
        print(f"  → {new_num_terms} output terms")
        
        if new_num_terms >= max_terms:
            raise RuntimeError(
                f"Pauli expansion term overflow: {new_num_terms} >= max_terms={max_terms}. "
                "max_terms를 늘리거나 회로의 rotation 수를 줄이세요.")
        
        # 현재 상태 디버깅: 정확한 buffer 포인터로 접근
        if current_input == input_expansion:
            buffer_pauli = d_input_pauli_buffer
            buffer_coef = d_input_coef_buffer
        else:
            buffer_pauli = d_output_pauli_buffer
            buffer_coef = d_output_coef_buffer
        
        pauli_data = buffer_pauli[:new_num_terms*2*num_packed_ints].get()
        coef_data = buffer_coef[:new_num_terms].get()
        
        for j in range(min(new_num_terms, 2)):  # 최대 5개 항만 출력
            x_mask = pauli_data[2*j]
            z_mask = pauli_data[2*j + 1]
            coef = coef_data[j]
            print(f"    Term {j}: X={x_mask:04b} Z={z_mask:04b} coef={coef:+.4f}")
        if new_num_terms > 5:
            print(f"    ... and {new_num_terms - 5} more terms")
        print()

    final_num_terms = cupp.pauli_expansion_get_num_terms(handle, current_input)
    print(f"✓ Propagation 완료: 최종 {final_num_terms}개 Pauli terms")
    e = time.time()
    print(f"총 소요 시간: {e - s:.2f} 초")


observable_evolution(handle, workspace, gate_ops, input_expansion=input_expansion, output_expansion=output_expansion, truncation_strategies=None)

Pauli propagation (역순, adjoint):
Observable을 circuit의 역방향으로 propagate...

Gate 4 (reverse): 22 input terms
  → 11 output terms
    Term 0: X=0100 Z=0010 coef=-0.0000
    Term 1: X=0111 Z=0010 coef=-0.0000
    ... and 6 more terms

Gate 3 (reverse): 11 input terms
  → 11 output terms
    Term 0: X=0100 Z=0010 coef=-0.0000
    Term 1: X=0111 Z=0010 coef=+0.0000
    ... and 6 more terms

Gate 2 (reverse): 11 input terms
  → 11 output terms
    Term 0: X=0100 Z=0010 coef=-0.0000
    Term 1: X=0111 Z=0010 coef=+0.0000
    ... and 6 more terms

Gate 1 (reverse): 11 input terms
  → 11 output terms
    Term 0: X=0100 Z=0010 coef=-0.0000
    Term 1: X=0111 Z=0010 coef=+0.0000
    ... and 6 more terms

✓ Propagation 완료: 최종 11개 Pauli terms
총 소요 시간: 0.00 초


### 기댓값 계산

**계산**: $\text{Tr}(|0\rangle\langle 0| \cdot observable)$  

이는 $\langle 0\vert observable \vert0\rangle$와 같습니다.

**API**: `pauli_expansion_view_compute_trace_with_zero_state`
- 입력: Propagated observable
- 출력: $\langle 0|O'|0\rangle$

In [ ]:
print("기댓값 계산...\n")

# 최종 view 생성
final_view = cupp.pauli_expansion_get_contiguous_range(
    handle, current_input, 0, final_num_terms
)

# 최종 Pauli expansion 내용 디버깅
print("최종 Pauli Expansion 상태:")
if current_input == input_expansion:
    buffer_pauli = d_input_pauli_buffer
    buffer_coef = d_input_coef_buffer
else:
    buffer_pauli = d_output_pauli_buffer
    buffer_coef = d_output_coef_buffer

pauli_data = buffer_pauli[:final_num_terms*2*num_packed_ints].get()
coef_data = buffer_coef[:final_num_terms].get()

for j in range(final_num_terms):
    x_mask = pauli_data[2*j]
    z_mask = pauli_data[2*j + 1]
    coef = coef_data[j]
    print(f"  Term {j}: X={x_mask:04b} Z={z_mask:04b} coef={coef:.6f}")

print()

# Trace 계산: ⟨0|evolved_observable|0⟩
result = np.array([0.0], dtype=np.float64)
cupp.pauli_expansion_view_compute_trace_with_zero_state(
    handle,
    final_view,
    result.ctypes.data,
    workspace  # workspace 포인터 주소
)

expectation_value = result[0]

print("=" * 60)
print(f"결과: ⟨ψ|O|ψ⟩ = {expectation_value:.6f}")
print("=" * 60)
